In [ ]:
import eikon as ek
import numpy as np
import calendar
import cufflinks as cf
import pandas as pd
from pandas.tseries.offsets import BMonthEnd
from datetime import datetime

cf.go_offline()

app_key = 'YOUR_APP_KEY'
ek.set_app_key(app_key)

def current_month():
    return datetime.now().month

def sys_year():
    today_date = datetime.strptime(datetime.today().strftime('%Y-%m-%d'), '%Y-%m-%d')
    y = str(today_date.year)[-2:]
    return y

def last_workday(d):
    last = BMonthEnd()
    return last.rollforward(d)

def eom(y, m):
    ed = last_workday(datetime(year = int(y), month=m, day=1)).strftime('%Y-%m-%d')
    return ed

def som(y, m):
    sd = datetime(year = int(y), month=m, day=1).strftime('%Y-%m-%d')
    return sd

def gen_contracts(r, d, m):
    xd, err = ek.get_data(r, 'EXPIR_DATE')
    xd = xd['EXPIR_DATE'][0]
    xd = xd.split('-')
    y = xd[0]
    _m = int(xd[1]) if m is None else m
    _y = int(y)
    _x = []
    x = []
    root = contract_data(r)
    while d >= 0:
        if d == 0:
            _x = ('{}{}'.format(root, str(_y - d)[-1:]))
            _ed = eom(_y-d, _m)
        else:
            dec = str(_y - d)[-2:][0]
            _x = '{}{}^{}'.format(root, str(_y - d)[-1], dec)
            _ed = eom(_y-d, _m)
        _sd = som(_y-d, _m)
        x.append([_x, _sd, _ed])
        d -= 1
    return x

def contract_data(r):
    k = list(r)
    root = ''.join(k[:-1])
    return root

def get_timeseries_seq(l):
    d = {}
    for i in l:
        d[i[0]] = ek.get_timeseries(i[0], 'CLOSE', start_date=i[1], end_date=i[2], interval='daily')
    return d

def get_data(ric, lookback_years = 5, month = current_month(), rebase = False):
    if 'c' in ric:
        return 'Please enter a futures delivery month contract code (e.g. LCOK0).'
    
    frames = gen_contracts(ric, lookback_years, month)
    hist = get_timeseries_seq(frames)
    
    for k in hist.keys():
        _idx = []
        for i in range(len(hist[k])):
            _idx.append(list(hist[k].index)[i].strftime('%b-%d'))
        hist[k]['Dates'] = _idx
    
    seasonal_df = pd.DataFrame(columns = ['Dates'])
    for k in hist.keys():
        seasonal_df = pd.merge(seasonal_df, hist[k], on='Dates', how = 'outer')
    
    seasonal_df = seasonal_df.sort_values(by='Dates')
    seasonal_df = seasonal_df.fillna(method='backfill')
    seasonal_df = seasonal_df.reset_index(drop = True)
    seasonal_df.index = seasonal_df['Dates']
    seasonal_df = seasonal_df.drop(columns='Dates')
    seasonal_df.columns = list(hist.keys())
    title = 'Seasonality chart'
    ytitle = 'Price'
    if rebase:
        seasonal_df = pd.DataFrame((np.log(seasonal_df)-np.log(seasonal_df.iloc[0])))*100
        title = 'Rebased seasonality chart'
        ytitle = 'Change, %'
    seasonal_df.iplot(title=title, yTitle = ytitle)
    return seasonal_df

